In [ ]:
'''
Name    : Priyak Thakkar
Program : Bayesian Network (Car Insurance from 'bnlearn' database in RStudio)
Date    : 13th Octomber 2021
Time    : 8:22 PM (California, San Jose)


Question 1 : Gibbs Sampling

Estimate the expected medical cost given evidence: ^M = Sigma(m) mP(M = m j Evidence),
where Evidence = (Age = Adult; RiskAversion = Normal; MakeModel = FamilySedan; Mileage = 20000; OwnCarCost = 10000; 
                OwnCarDamage = Moderate; CarValue = 20000; PropertyCost = 100000; LiabilityCost = 10000; YearsLicensed = Average). 
                
The conditional probabilities appear in 'insurance2-create.lisp'. Submit your answer and the code.

'''

In [99]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [114]:
model = BayesianNetwork([ 
                        ('SocioEcon','GoodStudent'),('Age','GoodStudent'),
                        ('Age','SocioEcon'),
                        ('Age','YearsLicensed'),
                        ('Age','RiskAversion'),('SocioEcon','RiskAversion'),
                        ('SocioEcon','VehicleYear'),('RiskAversion','VehicleYear'),
                        ('SocioEcon','MakeModel'), ('RiskAversion','MakeModel'),
                        ('Age','DrivingSkill'),('YearsLicensed','DrivingSkill'),
                        ('YearsLicensed','DrivingRecord'),('DrivingSkill','DrivingRecord'),('RiskAversion','DrivingRecord'),
                        ('RiskAversion','Garaged'),('SocioEcon','Garaged'), 
                        ('RiskAversion','AntiTheft'),('SocioEcon','AntiTheft'),
                        ('SocioEcon','ExtraCar'),
                        ('MakeModel','SafetyFeatures'),('VehicleYear','SafetyFeatures'),
                        ('DrivingSkill','DrivingBehavior'),('RiskAversion','DrivingBehavior'),
                        ('MakeModel','Ruggedness'),('VehicleYear','Ruggedness'),
                        ('MakeModel','Airbag'),('VehicleYear','Airbag'),
                        ('SafetyFeatures','Accident'),('Mileage','Accident'),('DrivingBehavior','Accident'),
                        ('MakeModel','CarValue'),('VehicleYear','CarValue'),('Mileage','CarValue'),
                        ('Accident','OwnCarDamage'),('Ruggedness','OwnCarDamage'),
                        ('AntiTheft','Theft'),('Garaged','Theft'),('CarValue','Theft'),
                        ('OwnCarDamage','OwnCarCost'),('CarValue','OwnCarCost'),('Theft','OwnCarCost'),
                        ('Ruggedness','Cushioning'),('Airbag','Cushioning'),
                        ('Accident','OtherCarCost'),('Ruggedness','OtherCarCost'),
                        ('OtherCarCost','PropertyCost'),('OwnCarCost','PropertyCost'),
                        ('Accident','MedicalCost'),('Age','MedicalCost'),('Cushioning','MedicalCost'),
                        ('Accident','LiabilityCost')
                        ])


In [115]:

cpd_age = TabularCPD(
                        variable='Age', 
                        variable_card=3, 
                        values=[[0.2],[0.6],[0.2]], 
                        state_names={
                                'Age':          ['Young','Adult','Senior']
                        })

cpd_socioEcon = TabularCPD(
                        variable='SocioEcon', variable_card=4, 
                        values=[
                                [0.40, 0.40, 0.50],
                                [0.40, 0.40, 0.20],
                                [0.19, 0.19, 0.29],
                                [0.01, 0.01, 0.01]
                        ],
                        evidence=['Age'],
                        evidence_card=[3], 
                        state_names={
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'], 
                                'Age':          ['Young','Adult','Senior']
                        })

cpd_yearsLicensed=TabularCPD(
                        variable= 'YearsLicensed',
                        variable_card= 3,
                        values=[
                                [0.9, 0.1, 0.01],      
                                [0.1, 0.6, 0.20],        
                                [0.0, 0.3, 0.79]
                        ],    
                        evidence =['Age'],
                        evidence_card=[3],    
                        state_names={
                                'YearsLicensed':['New','Average','Experienced'],     
                                'Age':          ['Young','Adult','Senior'],     
                        }
                        )

cpd_goodStudent = TabularCPD(   
                        variable='GoodStudent', 
                        variable_card= 2,
                        values=[
                                [0.1, 0.0, 0.0, 0.2, 0.0, 0.0, 0.5, 0.0, 0.0, 0.4, 0.0, 0.0],
                                [0.9, 1.0, 1.0, 0.8, 1.0, 1.0, 0.5, 1.0, 1.0, 0.6, 1.0, 1.0]
                        ],
                        evidence=['SocioEcon','Age'],
                        evidence_card=[4,3],
                        state_names= {
                                'GoodStudent':  ['True','False'], 
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'], 
                                'Age':          ['Young','Adult','Senior']
                        })

cpd_riskAversion =TabularCPD(
                        variable= 'RiskAversion',
                        variable_card= 4,
                        values=[
                                [0.02, 0.02, 0.02, 0.02, 0.015, 0.015, 0.015, 0.015, 0.01, 0.01, 0.01, 0.01],      
                                [0.58, 0.38, 0.48, 0.58, 0.285, 0.185, 0.285, 0.285, 0.09, 0.04, 0.09, 0.09],        
                                [0.30, 0.50, 0.40, 0.30, 0.500, 0.600, 0.500, 0.400, 0.40, 0.35, 0.40, 0.40],       
                                [0.10, 0.10, 0.10, 0.10, 0.200, 0.200, 0.200, 0.300, 0.50, 0.60, 0.50, 0.50]
                        ],    
                        evidence =['Age','SocioEcon'],
                        evidence_card=[3,4],    
                        state_names={
                                'RiskAversion': ['Maniac','Adventurous','Normal','Cautious'],     
                                'Age':          ['Young','Adult','Senior'],     
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'],
                        }
                        )

cpd_vehicleYear =TabularCPD(
                        variable= 'VehicleYear',
                        variable_card= 2,
                        values=[
                                [0.15, 0.15, 0.15, 0.15, 0.3, 0.3, 0.3, 0.3, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9],      
                                [0.85, 0.85, 0.85, 0.85, 0.7, 0.7, 0.7, 0.7, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1]
                        ],    
                        evidence =['SocioEcon','RiskAversion'],
                        evidence_card=[4,4],    
                        state_names={
                                'VehicleYear':  ['Current', 'Older'],
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'],
                                'RiskAversion': ['Maniac','Adventurous','Normal','Cautious']                
                        })

cpd_makeModel =TabularCPD(
                        variable= 'MakeModel',
                        variable_card= 5,
                        values=[
                                [0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.20, 0.20, 0.20, 0.20, 0.30, 0.30, 0.30, 0.30],      
                                [0.7, 0.7, 0.7, 0.7, 0.20, 0.20, 0.20, 0.20, 0.05, 0.05, 0.05, 0.05, 0.01, 0.01, 0.01, 0.01],        
                                [0.2, 0.2, 0.2, 0.2, 0.65, 0.65, 0.65, 0.65, 0.30, 0.30, 0.30, 0.30, 0.09, 0.09, 0.09, 0.09],       
                                [0.0, 0.0, 0.0, 0.0, 0.00, 0.00, 0.00, 0.00, 0.45, 0.45, 0.45, 0.45, 0.40, 0.40, 0.40, 0.40],
                                [0.0, 0.0, 0.0, 0.0, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.20, 0.20, 0.20, 0.20]
                        ],    
                        evidence =['SocioEcon','RiskAversion'],
                        evidence_card=[4,4],    
                        state_names={
                                'MakeModel':    ['SportsCar','Economy','FamilySedan','Luxury','SuperLuxury'],     
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'],     
                                'RiskAversion': ['Maniac','Adventurous','Normal','Cautious'],
                        }
                        )

cpd_drivingSkill =TabularCPD(
                        variable= 'DrivingSkill',
                        variable_card= 3,
                        values=[
                                [0.70, 0.50, 0.30, 0.40, 0.20, 0.10, 0.70, 0.30, 0.25],      
                                [0.25, 0.45, 0.60, 0.50, 0.60, 0.50, 0.25, 0.50, 0.60],        
                                [0.05, 0.05, 0.10, 0.10, 0.20, 0.40, 0.05, 0.20, 0.15]      
                        ],    
                        evidence =['Age','YearsLicensed'],
                        evidence_card=[3,3],    
                        state_names={
                                'DrivingSkill': ['SubStandard','Normal','Expert'],     
                                'Age':          ['Young','Adult','Senior'],     
                                'YearsLicensed':['New','Average','Experienced']
                        }
                        )

cpd_drivingRecord =TabularCPD(
                        variable= 'DrivingRecord',
                        variable_card= 3,
                        values=[
                                [0.02, 0.08, 0.20, 0.60, 0.10, 0.30, 0.55, 0.80, 0.20, 0.40, 0.70, 0.90, 0.10, 0.30, 0.55, 0.80, 0.20, 0.40, 0.70, 0.90, 0.30, 0.55, 0.80, 0.950, 0.20, 0.40, 0.70, 0.90, 0.30, 0.55, 0.80, 0.950, 0.40, 0.80, 0.950, 0.980],      
                                [0.28, 0.42, 0.60, 0.30, 0.60, 0.55, 0.35, 0.18, 0.60, 0.50, 0.25, 0.09, 0.60, 0.55, 0.35, 0.18, 0.60, 0.50, 0.25, 0.09, 0.60, 0.40, 0.18, 0.045, 0.60, 0.50, 0.25, 0.09, 0.60, 0.40, 0.18, 0.045, 0.55, 0.18, 0.045, 0.019],        
                                [0.70, 0.50, 0.20, 0.10, 0.30, 0.15, 0.10, 0.02, 0.20, 0.10, 0.05, 0.01, 0.30, 0.15, 0.10, 0.02, 0.20, 0.10, 0.05, 0.01, 0.10, 0.05, 0.02, 0.005, 0.20, 0.10, 0.05, 0.01, 0.10, 0.05, 0.02, 0.005, 0.05, 0.02, 0.005, 0.001]
                        ],    
                        evidence =['YearsLicensed','DrivingSkill','RiskAversion'],
                        evidence_card=[3,3,4],    
                        state_names={
                                'DrivingRecord':['Zero','One','Many'],     
                                'YearsLicensed':['New','Average','Experienced'],     
                                'DrivingSkill': ['SubStandard','Normal','Expert'],
                                'RiskAversion': ['Maniac','Adventurous','Normal','Cautious']
                        }
                        )

cpd_garaged =TabularCPD(
                        variable= 'Garaged',
                        variable_card= 4,
                        values=[
                                [0.000001, 0.15, 0.35, 0.489999, 0.000001, 0.01, 0.2, 0.950000, 0.000001, 0.299999, 0.500000, 0.850000, 0.000001, 0.950000, 0.999997, 0.999997],      
                                [0.800000, 0.80, 0.60, 0.500000, 0.800000, 0.25, 0.4, 0.000001, 0.800000, 0.000001, 0.000001, 0.000001, 0.800000, 0.000001, 0.000001, 0.000001],        
                                [0.049999, 0.04, 0.04, 0.000001, 0.050000, 0.60, 0.3, 0.000001, 0.050000, 0.600000, 0.400000, 0.001000, 0.050000, 0.024445, 0.000001, 0.000001],       
                                [0.150000, 0.01, 0.01, 0.010000, 0.149999, 0.14, 0.1, 0.049998, 0.149999, 0.100000, 0.099999, 0.148999, 0.149999, 0.025554, 0.000001, 0.000001]
                        ],    
                        evidence =['RiskAversion','SocioEcon'],
                        evidence_card=[4,4],    
                        state_names={
                                'Garaged':      ['Secure','City','Suburb','Rural'],
                                'RiskAversion': ['Maniac','Adventurous','Normal','Cautious'],     
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'],
                        }
                        )

cpd_antiTheft =TabularCPD(
                        variable= 'AntiTheft',
                        variable_card= 2,
                        values=[
                                [0.000001, 0.000001, 0.05, 0.5, 0.000001, 0.000001, 0.2, 0.5, 0.1, 0.3, 0.9, 0.8, 0.95, 0.999999, 0.999999, 0.999999],      
                                [0.999999, 0.999999, 0.95, 0.5, 0.999999, 0.999999, 0.8, 0.5, 0.9, 0.7, 0.1, 0.2, 0.05, 0.000001, 0.000001, 0.000001]
                        ],    
                        evidence =['RiskAversion','SocioEcon'],
                        evidence_card=[4,4],    
                        state_names={
                                'AntiTheft':    ['True', 'False'],     
                                'RiskAversion': ['Maniac','Adventurous','Normal','Cautious'],     
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'],
                        }
                        )

cpd_extraCar =TabularCPD(
                        variable= 'ExtraCar',
                        variable_card= 2,
                        values=[
                                [0.5, 0.8, 0.9, 0.95],      
                                [0.5, 0.2, 0.1, 0.05]
                        ], 
                        evidence =['SocioEcon'],
                        evidence_card=[4],    
                        state_names={
                                'ExtraCar':     ['True','False'],
                                'RiskAversion': ['True', 'False'],     
                                'SocioEcon':    ['Lower','Middle','UpperMiddle','Wealthy'],
                        }
                        )

cpd_safetyFeatures =TabularCPD(
                        variable= 'SafetyFeatures',
                        variable_card= 2,
                        values=[
                                [0.9, 0.1, 0.001, 0.0, 0.4, 0.0, 0.99, 0.3, 0.99, 0.15],      
                                [0.1, 0.9, 0.999, 1.0, 0.6, 1.0, 0.01, 0.7, 0.01, 0.85]       
                        ],    
                        evidence =['MakeModel','VehicleYear'],
                        evidence_card=[5,2],    
                        state_names={
                                'SafetyFeatures':       ['True','False'],     
                                'MakeModel':            ['SportsCar','Economy','FamilySedan','Luxury','SuperLuxury'],     
                                'VehicleYear':          ['Current','Older']
                        }
                        )

cpd_drivingBehavior =TabularCPD(
                        variable= 'DrivingBehavior',
                        variable_card= 3,
                        values=[
                                [1.0, 1.0, 1.0, 1.0, 0.5, 0.3, 0.0, 0.0, 0.3, 0.01, 0.0, 0.0],      
                                [0.0, 0.0, 0.0, 0.0, 0.2, 0.4, 1.0, 0.8, 0.2, 0.01, 0.0, 0.0],        
                                [0.0, 0.0, 0.0, 0.0, 0.3, 0.3, 0.0, 0.2, 0.5, 0.98, 1.0, 1.0]
                        ],    
                        evidence =['DrivingSkill','RiskAversion'],
                        evidence_card=[3,4],    
                        state_names={
                                'DrivingBehavior':      ['Poor','Normal','Excellent'],     
                                'DrivingSkill':         ['SubStandard','Normal','Expert'],     
                                'RiskAversion':         ['Maniac','Adventurous','Normal','Cautious']
                        }
                        )

cpd_ruggedness =TabularCPD(
                        variable= 'Ruggedness',
                        variable_card= 3,
                        values=[
                                [0.95, 0.95, 0.5, 0.9, 0.2, 0.05, 0.1, 0.1, 0.05, 0.05],      
                                [0.04, 0.04, 0.5, 0.1, 0.6, 0.55, 0.6, 0.6, 0.55, 0.55],        
                                [0.01, 0.01, 0.0, 0.0, 0.2, 0.40, 0.3, 0.3, 0.40, 0.40]
                        ],    
                        evidence =['MakeModel','VehicleYear'],
                        evidence_card=[5,2],    
                        state_names={
                                'Ruggedness':   ['TinCan', 'Normal', 'Tank'],     
                                'MakeModel':    ['SportsCar','Economy','FamilySedan', 'Luxury', 'SuperLuxury'],     
                                'VehicleYear':  ['Current', 'Older'],
                        }
                        )

cpd_airbag =TabularCPD(
                        variable= 'Airbag',
                        variable_card= 2,
                        values=[
                                [1.0, 0.1, 1.0, 0.05, 1.0, 0.2, 1.0, 0.6, 1.0, 0.1],      
                                [0.0, 0.9, 0.0, 0.95, 0.0, 0.8, 0.0, 0.4, 0.0, 0.9]
                        ],    
                        evidence =['MakeModel','VehicleYear'],
                        evidence_card=[5,2],    
                        state_names={
                                'Airbag':       ['True','False'],     
                                'MakeModel':    ['SportsCar', 'Economy','FamilySedan','Luxury','SuperLuxury'],     
                                'VehicleYear':  ['Current','Older'],
                        }
                        )

cpd_mileage = TabularCPD(
                        variable='Mileage', 
                        variable_card=4, 
                        values=[[0.1],[0.4],[0.4],[0.1]], 
                        state_names={
                                'Mileage':       ['FiveThou','TwentyThou','FiftyThou','Extreme']
                        })

cpd_accident =TabularCPD(
                        variable= 'Accident',
                        variable_card= 4,
                        values=[
                                [0.70, 0.990, 0.9990, 0.4, 0.980, 0.995, 0.3, 0.970, 0.990, 0.2, 0.95, 0.985, 0.6, 0.980, 0.995, 0.3, 0.960, 0.990, 0.2, 0.950, 0.980, 0.1, 0.94, 0.980],      
                                [0.20, 0.007, 0.0007, 0.3, 0.010, 0.003, 0.3, 0.020, 0.007, 0.2, 0.03, 0.010, 0.2, 0.010, 0.003, 0.2, 0.020, 0.007, 0.2, 0.030, 0.010, 0.1, 0.03, 0.010],        
                                [0.07, 0.002, 0.0002, 0.2, 0.005, 0.001, 0.2, 0.007, 0.002, 0.3, 0.01, 0.003, 0.1, 0.005, 0.001, 0.2, 0.015, 0.002, 0.2, 0.015, 0.005, 0.3, 0.02, 0.007],       
                                [0.03, 0.001, 0.0001, 0.1, 0.005, 0.001, 0.2, 0.003, 0.001, 0.3, 0.01, 0.002, 0.1, 0.005, 0.001, 0.3, 0.005, 0.001, 0.4, 0.005, 0.005, 0.5, 0.01, 0.003]
                        ],    
                        evidence =['SafetyFeatures','Mileage','DrivingBehavior'],
                        evidence_card=[2,4,3],    
                        state_names={
                                'Accident':             ['None', 'Mild', 'Moderate','Severe'],
                                'SafetyFeatures':       ['True','False'],
                                'Mileage':              ['FiveThou','TwentyThou','FiftyThou', 'Extreme'],     
                                'DrivingBehavior':      ['Poor','Normal','Excellent'],
                        }
                        )

cpd_carValue =TabularCPD(
                        variable= 'CarValue',
                        variable_card= 5,
                        values=[
                                [0.00, 0.00, 0.00, 0.00, 0.03, 0.16, 0.40, 0.90, 0.1, 0.1, 0.1, 0.1, 0.25, 0.7000, 0.990000, 0.999998, 0.0, 0.0, 0.0, 0.0, 0.2, 0.5, 0.7, 0.990000, 0.0, 0.0, 0.0, 0.0, 0.01, 0.05, 0.1, 0.2, 0.0, 0.0, 0.0, 0.0, 0.000001, 0.000001, 0.000001, 0.000001],      
                                [0.10, 0.10, 0.10, 0.10, 0.30, 0.50, 0.47, 0.06, 0.8, 0.8, 0.8, 0.8, 0.70, 0.2999, 0.009999, 0.000001, 0.1, 0.1, 0.1, 0.1, 0.3, 0.3, 0.2, 0.009999, 0.0, 0.0, 0.0, 0.0, 0.09, 0.15, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.000001, 0.000001, 0.000001, 0.000001],        
                                [0.80, 0.80, 0.80, 0.80, 0.60, 0.30, 0.10, 0.02, 0.1, 0.1, 0.1, 0.1, 0.05, 0.0001, 0.000001, 0.000001, 0.9, 0.9, 0.9, 0.9, 0.5, 0.2, 0.1, 0.000001, 0.0, 0.0, 0.0, 0.0, 0.20, 0.30, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0, 0.000001, 0.000001, 0.000001, 0.000001],       
                                [0.09, 0.09, 0.09, 0.09, 0.06, 0.03, 0.02, 0.01, 0.0, 0.0, 0.0, 0.0, 0.00, 0.0000, 0.000000, 0.000000, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000000, 1.0, 1.0, 1.0, 1.0, 0.70, 0.50, 0.3, 0.3, 0.0, 0.0, 0.0, 0.0, 0.000001, 0.000001, 0.000001, 0.000001],
                                [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.00, 0.0000, 0.000000, 0.000000, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000000, 0.0, 0.0, 0.0, 0.0, 0.00, 0.00, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.999996, 0.999996, 0.999996, 0.999996]
                        ],    
                        evidence =['MakeModel','VehicleYear','Mileage'],
                        evidence_card=[5,2,4],    
                        state_names={
                                'CarValue' :    ['FiveThou','TenThou','TwentyThou','FiftyThou','Million'],     
                                'MakeModel':    ['SportsCar','Economy','FamilySedan','Luxury','SuperLuxury'],     
                                'VehicleYear':  ['Current','Older'],
                                'Mileage':      ['FiveThou','TwentyThou','FiftyThou','Extreme']
                        }
                        )

cpd_ownCarDamage =TabularCPD(
                        variable= 'OwnCarDamage',
                        variable_card= 4,
                        values=[
                                [1.0, 1.0, 1.0, 0.001, 0.200000, 0.700000, 0.000001, 0.001, 0.05, 0.000001, 0.000001, 0.05],      
                                [0.0, 0.0, 0.0, 0.900, 0.750000, 0.290000, 0.000999, 0.099, 0.60, 0.000009, 0.000999, 0.20],        
                                [0.0, 0.0, 0.0, 0.098, 0.049999, 0.009999, 0.700000, 0.800, 0.30, 0.000090, 0.009000, 0.20],       
                                [0.0, 0.0, 0.0, 0.001, 0.000001, 0.000001, 0.299000, 0.100, 0.05, 0.999900, 0.990000, 0.55]
                        ],    
                        evidence =['Accident','Ruggedness'],
                        evidence_card=[4,3],    
                        state_names={
                                'OwnCarDamage': ['None','Mild','Moderate','Severe'],     
                                'Accident':     ['None','Mild','Moderate','Severe'],    
                                'Ruggedness':   ['TinCan', 'Normal', 'Tank']
                        }
                        )

cpd_theft =TabularCPD(
                        variable= 'Theft',
                        variable_card= 2,
                        values=[
                                [0.000001, 0.000002, 0.000003, 0.000002, 0.000001, 0.0005, 0.002, 0.005, 0.005, 0.000001, 0.000001, 0.0001, 0.0003, 0.0003, 0.000001, 0.00001, 0.000002, 0.00005, 0.00005, 0.000001, 0.000001, 0.000002, 0.000003, 0.000002, 0.000001, 0.001, 0.005, 0.01, 0.01, 0.000001, 0.00001, 0.0002, 0.0005, 0.0005, 0.000001, 0.00001, 0.0001, 0.0002, 0.0002, 0.000001],      
                                [0.999999, 0.999998, 0.999997, 0.999998, 0.999999, 0.9995, 0.998, 0.995, 0.995, 0.999999, 0.999999, 0.9999, 0.9997, 0.9997, 0.999999, 0.99999, 0.999998, 0.99995, 0.99995, 0.999999, 0.999999, 0.999998, 0.999997, 0.999998, 0.999999, 0.999, 0.995, 0.99, 0.99, 0.999999, 0.99999, 0.9998, 0.9995, 0.9995, 0.999999, 0.99999, 0.9999, 0.9998, 0.9998, 0.999999]       
                        ],    
                        evidence =['AntiTheft','Garaged','CarValue'],
                        evidence_card=[2,4,5],    
                        state_names={
                                'Theft':        ['True','False'],     
                                'AntiTheft':    ['True','False'],     
                                'Garaged':      ['Secure','City','Suburb','Rural'],
                                'CarValue' :    ['FiveThou','TenThou','TwentyThou','FiftyThou','Million']
                        }
                        )

cpd_ownCarCost =TabularCPD(
                        variable= 'OwnCarCost',
                        variable_card= 4,
                        values=[
                                [0.2, 1.0, 0.05, 1.0, 0.04, 1.0, 0.04, 1.0, 0.04, 1.0, 0.15, 0.95, 0.03, 0.95, 0.03, 0.99, 0.03, 0.99, 0.02, 0.98, 0.05, 0.25, 0.01, 0.15, 0.001, 0.01, 0.001, 0.005, 0.001, 0.003, 0.03, 0.05, 0.000001, 0.01, 0.000001, 0.005, 0.000001, 0.001, 0.000001, 0.000001],      
                                [0.8, 0.0, 0.95, 0.0, 0.01, 0.0, 0.01, 0.0, 0.01, 0.0, 0.85, 0.05, 0.97, 0.05, 0.02, 0.01, 0.02, 0.01, 0.03, 0.01, 0.95, 0.75, 0.99, 0.85, 0.001, 0.01, 0.001, 0.005, 0.001, 0.003, 0.97, 0.95, 0.999999, 0.99, 0.000001, 0.005, 0.000001, 0.001, 0.000001, 0.000001],        
                                [0.0, 0.0, 0.00, 0.0, 0.95, 0.0, 0.95, 0.0, 0.20, 0.0, 0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.95, 0.00, 0.25, 0.01, 0.00, 0.00, 0.00, 0.00, 0.998, 0.98, 0.998, 0.990, 0.018, 0.044, 0.00, 0.00, 0.000000, 0.00, 0.999998, 0.990, 0.999998, 0.998, 0.009998, 0.029998],       
                                [0.0, 0.0, 0.00, 0.0, 0.00, 0.0, 0.00, 0.0, 0.75, 0.0, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.000, 0.00, 0.000, 0.000, 0.980, 0.950, 0.00, 0.00, 0.000000, 0.00, 0.000000, 0.000, 0.000000, 0.000, 0.990000, 0.970000]
                        ],    
                        evidence =['OwnCarDamage','CarValue','Theft'],
                        evidence_card=[4,5,2],    
                        state_names={
                                'OwnCarCost' :  ['Thousand','TenThou','HundredThou','Million'],
                                'OwnCarDamage': ['None','Mild','Moderate','Severe'],     
                                'CarValue':     ['FiveThou','TenThou','TwentyThou','FiftyThou','Million'],     
                                'Theft':        ['True','False']
                        }
                        )

cpd_cushioning =TabularCPD(
                        variable= 'Cushioning',
                        variable_card= 4,
                        values=[
                                [0.5, 0.7, 0.0, 0.1, 0.0, 0.0],      
                                [0.3, 0.3, 0.1, 0.6, 0.0, 0.0],        
                                [0.2, 0.0, 0.6, 0.3, 0.0, 0.7],       
                                [0.0, 0.0, 0.3, 0.0, 1.0, 0.3]
                        ],    
                        evidence =['Ruggedness','Airbag'],
                        evidence_card=[3,2],    
                        state_names={
                                'Cushioning':   ['Poor','Fair','Good','Excellent'],    
                                'Ruggedness':   ['TinCan','Normal','Tank'],     
                                'Airbag':       ['True','False']
                        }
                        )

cpd_otherCarCost =TabularCPD(
                        variable= 'OtherCarCost',
                        variable_card= 4,
                        values=[
                                [1.0, 1.0, 1.0, 0.99000, 9.799657e-01, 0.95000, 0.60000, 0.50000, 0.40000, 0.20000, 0.10000, 0.0050],      
                                [0.0, 0.0, 0.0, 0.00500, 9.999650e-03, 0.03000, 0.20000, 0.20000, 0.30000, 0.40000, 0.50000, 0.5500],        
                                [0.0, 0.0, 0.0, 0.00499, 9.984651e-03, 0.01998, 0.19998, 0.29997, 0.29996, 0.39996, 0.39994, 0.4449],       
                                [0.0, 0.0, 0.0, 0.00001, 4.999825e-05, 0.00002, 0.00002, 0.00003, 0.00004, 0.00004, 0.00006, 0.0001]
                        ],    
                        evidence =['Accident','Ruggedness'],
                        evidence_card=[4,3],    
                        state_names={
                                'OtherCarCost': ['Thousand','TenThou','HundredThou','Million'],     
                                'Accident':     ['None','Mild','Moderate','Severe'],     
                                'Ruggedness':   ['TinCan','Normal','Tank']
                        }
                        )

cpd_propertyCost =TabularCPD(
                        variable= 'PropertyCost',
                        variable_card= 4,
                        values=[
                                [0.7, 0.00, 0.00, 0.0, 0.00, 0.0, 0.00, 0.0, 0.00, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],      
                                [0.3, 0.95, 0.00, 0.0, 0.95, 0.6, 0.00, 0.0, 0.00, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],        
                                [0.0, 0.05, 0.98, 0.0, 0.05, 0.4, 0.95, 0.0, 0.98, 0.8, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0],
                                [0.0, 0.00, 0.02, 1.0, 0.00, 0.0, 0.05, 1.0, 0.02, 0.2, 0.4, 1.0, 1.0, 1.0, 1.0, 1.0]      
                        ],    
                        evidence =['OtherCarCost','OwnCarCost'],
                        evidence_card=[4,4],    
                        state_names={
                                'PropertyCost': ['Thousand','TenThou','HundredThou','Million'],     
                                'OtherCarCost': ['Thousand','TenThou','HundredThou','Million'],     
                                'OwnCarCost':   ['Thousand','TenThou','HundredThou','Million']
                        }
                        )

cpd_medicalCost =TabularCPD(
                        variable= 'MedicalCost',
                        variable_card= 4,
                        values=[
                                [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.960, 0.9800, 0.99000, 0.999000, 0.960, 0.9800, 0.99000, 0.999000, 0.90, 0.950, 0.970, 0.990, 0.5, 0.80, 0.95, 0.990, 0.5, 0.80, 0.95, 0.990, 0.3, 0.5, 0.90, 0.95, 0.3, 0.5, 0.90, 0.95, 0.3, 0.5, 0.90, 0.95, 0.2, 0.3, 0.60, 0.90],      
                                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030, 0.0190, 0.00990, 0.000990, 0.030, 0.0190, 0.00990, 0.000990, 0.07, 0.040, 0.025, 0.007, 0.2, 0.15, 0.02, 0.007, 0.2, 0.15, 0.02, 0.007, 0.3, 0.2, 0.07, 0.03, 0.3, 0.2, 0.07, 0.03, 0.3, 0.2, 0.07, 0.03, 0.2, 0.3, 0.30, 0.05],        
                                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009, 0.0009, 0.00009, 0.000009, 0.009, 0.0009, 0.00009, 0.000009, 0.02, 0.007, 0.003, 0.002, 0.2, 0.03, 0.02, 0.002, 0.2, 0.03, 0.02, 0.002, 0.2, 0.2, 0.02, 0.01, 0.2, 0.2, 0.02, 0.01, 0.2, 0.2, 0.02, 0.01, 0.3, 0.2, 0.07, 0.03],       
                                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0001, 0.00001, 0.000001, 0.001, 0.0001, 0.00001, 0.000001, 0.01, 0.003, 0.002, 0.001, 0.1, 0.02, 0.01, 0.001, 0.1, 0.02, 0.01, 0.001, 0.2, 0.1, 0.01, 0.01, 0.2, 0.1, 0.01, 0.01, 0.2, 0.1, 0.01, 0.01, 0.3, 0.2, 0.03, 0.02], 
                        ],    
                        evidence =['Accident','Age','Cushioning'],
                        evidence_card=[4,3,4],    
                        state_names={
                                'MedicalCost':  ['Thousand','TenThou','HundredThou','Million'],
                                'Accident':     ['None','Mild','Moderate','Severe'],     
                                'Age':          ['Young','Adult','Senior'],     
                                'Cushioning':   ['Poor','Fair','Good','Excellent']
                        }
                        )

cpd_liabilityCost =TabularCPD(
                        variable= 'LiabilityCost',
                        variable_card= 4,
                        values=[
                                [1.0, 0.999000, 0.90, 0.80],      
                                [0.0, 0.000998, 0.05, 0.10],        
                                [0.0, 0.000001, 0.03, 0.06],       
                                [0.0, 0.000001, 0.02, 0.04]
                        ],    
                        evidence =['Accident'],
                        evidence_card=[4],    
                        state_names={
                                'LiabilityCost': ['Thousand','TenThou','HundredThou','Million'],     
                                'Accident':      ['None','Mild','Moderate','Severe'],
                        }
                        )



True

In [116]:
model.add_cpds( cpd_age, cpd_socioEcon, cpd_goodStudent, cpd_riskAversion, cpd_vehicleYear, cpd_makeModel, cpd_yearsLicensed, cpd_drivingSkill, cpd_drivingRecord, cpd_garaged,
                cpd_antiTheft, cpd_extraCar, cpd_safetyFeatures, cpd_drivingBehavior, cpd_ruggedness, cpd_airbag, cpd_mileage, cpd_accident, cpd_carValue, cpd_ownCarDamage, cpd_theft,
                cpd_ownCarCost, cpd_cushioning, cpd_otherCarCost, cpd_propertyCost, cpd_medicalCost, cpd_liabilityCost)
model.check_model()

True

In [117]:
print(cpd_socioEcon)
print(cpd_mileage)

+------------------------+------------+------------+-------------+
| Age                    | Age(Young) | Age(Adult) | Age(Senior) |
+------------------------+------------+------------+-------------+
| SocioEcon(Lower)       | 0.4        | 0.4        | 0.5         |
+------------------------+------------+------------+-------------+
| SocioEcon(Middle)      | 0.4        | 0.4        | 0.2         |
+------------------------+------------+------------+-------------+
| SocioEcon(UpperMiddle) | 0.19       | 0.19       | 0.29        |
+------------------------+------------+------------+-------------+
| SocioEcon(Wealthy)     | 0.01       | 0.01       | 0.01        |
+------------------------+------------+------------+-------------+
+---------------------+-----+
| Mileage(FiveThou)   | 0.1 |
+---------------------+-----+
| Mileage(TwentyThou) | 0.4 |
+---------------------+-----+
| Mileage(FiftyThou)  | 0.4 |
+---------------------+-----+
| Mileage(Extreme)    | 0.1 |
+---------------------+

In [118]:
print(model.get_cpds('SocioEcon'))

+------------------------+------------+------------+-------------+
| Age                    | Age(Young) | Age(Adult) | Age(Senior) |
+------------------------+------------+------------+-------------+
| SocioEcon(Lower)       | 0.4        | 0.4        | 0.5         |
+------------------------+------------+------------+-------------+
| SocioEcon(Middle)      | 0.4        | 0.4        | 0.2         |
+------------------------+------------+------------+-------------+
| SocioEcon(UpperMiddle) | 0.19       | 0.19       | 0.29        |
+------------------------+------------+------------+-------------+
| SocioEcon(Wealthy)     | 0.01       | 0.01       | 0.01        |
+------------------------+------------+------------+-------------+


In [119]:
model.local_independencies(['Age', 'SocioEcon', 'GoodStudent', 'RiskAversion', 'VehicleYear', 'MakeModel', 'YearsLicensed', 'DrivingSkill', 
                            'DrivingRecord', 'Garaged', 'AntiTheft', 'ExtraCar', 'SafetyFeatures', 'DrivingBehavior', 'Ruggedness', 'Airbag', 'Mileage', 
                            'Accident', 'CarValue', 'OwnCarDamage', 'Theft', 'OwnCarCost', 'Cushioning', 'OtherCarCost', 'PropertyCost', 'MedicalCost', 'LiabilityCost'])

(Age ⟂ Mileage)
(SocioEcon ⟂ Mileage, DrivingSkill, YearsLicensed | Age)
(GoodStudent ⟂ PropertyCost, AntiTheft, YearsLicensed, RiskAversion, DrivingRecord, VehicleYear, LiabilityCost, OwnCarCost, OwnCarDamage, Ruggedness, MakeModel, Garaged, MedicalCost, DrivingBehavior, Accident, OtherCarCost, DrivingSkill, Mileage, Cushioning, SafetyFeatures, CarValue, Theft, Airbag, ExtraCar | Age, SocioEcon)
(RiskAversion ⟂ YearsLicensed, ExtraCar, Mileage, DrivingSkill, GoodStudent | Age, SocioEcon)
(VehicleYear ⟂ MakeModel, AntiTheft, Garaged, YearsLicensed, DrivingBehavior, Mileage, DrivingSkill, DrivingRecord, ExtraCar, Age, GoodStudent | RiskAversion, SocioEcon)
(MakeModel ⟂ AntiTheft, Garaged, YearsLicensed, DrivingBehavior, Mileage, DrivingSkill, DrivingRecord, VehicleYear, ExtraCar, Age, GoodStudent | RiskAversion, SocioEcon)
(YearsLicensed ⟂ MakeModel, AntiTheft, Garaged, RiskAversion, Mileage, SocioEcon, Cushioning, VehicleYear, SafetyFeatures, CarValue, Theft, Airbag, ExtraCar, GoodStud

In [120]:
model.local_independencies(['MedicalCost'])

(MedicalCost ⟂ PropertyCost, AntiTheft, YearsLicensed, RiskAversion, DrivingRecord, VehicleYear, LiabilityCost, OwnCarCost, OwnCarDamage, GoodStudent, Ruggedness, MakeModel, Garaged, DrivingBehavior, Mileage, OtherCarCost, DrivingSkill, SafetyFeatures, CarValue, Theft, Airbag, ExtraCar, SocioEcon | Cushioning, Age, Accident)

In [121]:
model.active_trail_nodes('MedicalCost')

{'MedicalCost': {'Accident',
  'Age',
  'Airbag',
  'AntiTheft',
  'CarValue',
  'Cushioning',
  'DrivingBehavior',
  'DrivingRecord',
  'DrivingSkill',
  'ExtraCar',
  'Garaged',
  'GoodStudent',
  'LiabilityCost',
  'MakeModel',
  'MedicalCost',
  'Mileage',
  'OtherCarCost',
  'OwnCarCost',
  'OwnCarDamage',
  'PropertyCost',
  'RiskAversion',
  'Ruggedness',
  'SafetyFeatures',
  'SocioEcon',
  'Theft',
  'VehicleYear',
  'YearsLicensed'}}

In [122]:
infer = VariableElimination(model)
gs_dist = infer.query(['GoodStudent'])
print(gs_dist)

Eliminating: Age: 100%|██████████| 2/2 [00:00<00:00, 499.89it/s]

+--------------------+--------------------+
| GoodStudent        |   phi(GoodStudent) |
+====================+====================+
| GoodStudent(True)  |             0.0438 |
+--------------------+--------------------+
| GoodStudent(False) |             0.9562 |
+--------------------+--------------------+


In [123]:
print(infer.query(['MedicalCost'], evidence={'Age': 'Adult', 'RiskAversion': 'Normal', 'MakeModel': 'FamilySedan', 'Mileage': 'TwentyThou', 'OwnCarCost': 'TenThou',
                                    'OwnCarDamage': 'Moderate', 'CarValue': 'TwentyThou', 'PropertyCost': 'HundredThou', 'LiabilityCost': 'TenThou', 'YearsLicensed': 'Average'}))

Eliminating: Accident: 100%|██████████| 13/13 [00:00<00:00, 276.59it/s]

+--------------------------+--------------------+
| MedicalCost              |   phi(MedicalCost) |
+==========================+====================+
| MedicalCost(Thousand)    |             0.9005 |
+--------------------------+--------------------+
| MedicalCost(TenThou)     |             0.0549 |
+--------------------------+--------------------+
| MedicalCost(HundredThou) |             0.0286 |
+--------------------------+--------------------+
| MedicalCost(Million)     |             0.0160 |
+--------------------------+--------------------+
